# Evaluación python 1
## solo para ejercitar (el que dió la profe)

## PROBLEMA 01
En farmacocinética, la concentración plasmática de un fármaco se estudia para garantizar que el paciente tenga niveles terapéuticos seguros. Supongamos que la concentración del fármaco en el torrente sanguíneo C(t) (en mg/L) sigue el modelo:
$$ \frac{dC}{dt}=D - \lambda C \quad ; \quad C(0)=C$$
donde
* D>0 es la tasa de administración del fármaco, expresada en mg/L*h, considerada constante debido al uso de una bomba de infusión intravenosa
* $\lambda$ > 0 es la constante de eliminación metabólica y renal del organismo, expresada en $h^{-1}$
* $C_0$ es la concentración inicial del fármaco en plasma (mg/L)
* La variable t representa el tiempo medido en horas (h).
Este modelo lineal simplificado corresponde a una situación de infusión intravenosa continua, donde la entrada del fármaco es constante y la eliminación es proporcional a la concentradio.

**(a)** Mediante un análisis cualitativo de la ecuación diferencial (sin resolverla). Determine para qué valores de la concentración inicial $C_0$ la solución C(t) será:
* creciente
* decreciente
Interprete los resultados en térmicos clínicos: ¿qué significa que la concentración del fármaco aumente o disminuya en la sangre tras iniciar la infusión?

**(b)** Calcule la concentración de equilibrio y explique por qué este valor corresponde a la concentración plamsmática en estado estacionario. Discuta la importancia de alcanzar dicho estado en tratamientos de larga duración y los riesgos de desviarse de este equilibrio (toxicidad o ineficacia).

## PROBLEMA 02
En un sistema de ventilación mecánica, un pulmón artificial de volumen máximo $V$ (litros) recibe mezcla gaseosa desde un compresor $A$ a razón de $a$ L/min confracción inspirada en oxígeno $ c \in (0, 1) $ . Simultáneamente, el paciente exhala a razón de $b$ L/min hacía el exterior $B$. Sea $x(t)$ la cantidad de oxígeno almacenado en el pulmón artificial (en litros de $O_2$) y $V(T) = V - (b-a)t$ el volumen de gas dentro del pulmón (suponiendo $b > a$).
Además del arrastre por el flujo de salida, existe un consumo metabólico proporcional a la fracción de oxígeno en el pulmón, modelado con coeficiente $k > 0$ (min$^{-1}$). El balance de masa de $O_2$ queda:
$$ \frac{dx}{dt}=ca - \frac{bx}{V(t)}- \frac{kx}{V(t)} \quad ; \quad V(t) = V - (b-a)t$$
Aquí t está en minutos.
Use los parámetros: $V=600, a=5, b=6, c=0.21, k=2$ y condición inicial $x(0) = 120$

**(a)** Estime $x(30)$ con **Euler** y **RK4** usando $h=0,1$.

**(b)** Obtenga la solución exca en PYTHON (incluya el factor integrante explícito) y calcule el error absoluto de cada método respecto de la solución exacta y discuta si es aceptable para control de soporte ventilatorio.